# Dask Adaptive Scaling

In [1]:
from ncar_jobqueue import NCARCluster

In [3]:
cluster = NCARCluster(
    cores=2,
    processes=1,
    queue="casper",
    walltime="00:30:00",
    resource_spec="select=1:ncpus=1:mem=10GB",
)
cluster

/glade/work/abanihi/opt/miniconda/envs/playground/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39026 instead
  warnings.warn(


## Batch Job Script

In [4]:
print(cluster.job_script())

#!/usr/bin/env bash

#PBS -N dask-worker-casper-dav
#PBS -q casper
#PBS -A NIOW0001
#PBS -l select=1:ncpus=1:mem=10GB
#PBS -l walltime=00:30:00
#PBS -e /glade/scratch/abanihi/dask/casper-dav/logs/
#PBS -o /glade/scratch/abanihi/dask/casper-dav/logs/

/glade/work/abanihi/opt/miniconda/envs/playground/bin/python -m distributed.cli.dask_worker tcp://10.12.206.47:42170 --nthreads 2 --memory-limit 93.13GiB --name dummy-name --nanny --death-timeout 60 --local-directory /glade/scratch/abanihi/dask/casper-dav/local-dir --interface ib0 --protocol tcp://



## Cluster Scaling APIs

In [5]:
cluster.scale(5)

In [6]:
cluster.scale(jobs=1)

In [31]:
cluster.adapt(minimum_jobs=1, maximum_jobs=20)

## Connect cluster to the client

In [8]:
from distributed import Client

client = Client(cluster)

## Run some computation

In [32]:
import dask.array as da

x = da.random.random((200, 10_000, 5_000), chunks=(20, 1_000, 1_000))
x

dask.array<random_sample, shape=(200, 10000, 5000), dtype=float64, chunksize=(20, 1000, 1000), chunktype=numpy.ndarray>

In [33]:
y = x.std(axis=0)
y = y.persist()
y

dask.array<_sqrt, shape=(10000, 5000), dtype=float64, chunksize=(1000, 1000), chunktype=numpy.ndarray>